<a href="https://colab.research.google.com/github/ryandale7/ML-on-Graphs/blob/main/Machine_Learning_on_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **What is a Graph Neural Network?**

**GNNs** are a type of neural network specifically designed to work with **graph-structured data**. Unlike traditional neural networks, which process structured inputs like sequences or grids (e.g., text or images), GNNs can handle **nodes** (entities) and **edges** (relationships) in graphs, making them suitable for problems where data is best represented as a graph.


A graph \( G \) is a mathematical structure represented as:  
\[
G = (V, E)
\]  
where:
- \( V \): Set of **nodes** (or vertices), representing entities.
- \( E \): Set of **edges**, representing relationships between entities.
- \( A \): **Adjacency matrix** describing connections between nodes. \( A_{ij} = 1 \) if an edge exists between nodes \( i \) and \( j \).

Each node \( v \in V \) or edge \( e_{ij} \in E \) can also carry **features**. For example:
- In a **social network**, nodes are people, edges are friendships, and node features could include user attributes (age, interests, etc.).
- In a **molecular graph**, nodes are atoms, edges are chemical bonds, and node features represent atom types.

## **Step 1: Dependencies**
We will only use `numpy` for numerical operations. Install it if necessary:

In [8]:
pip install numpy

## **Step 2: Define the Graph Structure**
We represent the graph as an adjacency matrix \( A \), node features \( X \), and labels \( Y \).



In [9]:
import numpy as np

# Adjacency Matrix (undirected graph with 4 nodes)
A = np.array([
    [0, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0]
])

# Node features (4 nodes, 3 features each)
X = np.array([
    [1, 2, 1],
    [2, 1, 0],
    [3, 1, 4],
    [4, 2, 1]
])

# Node labels (for classification, e.g., binary)
Y = np.array([0, 1, 0, 1])  # Ground truth labels


## **Why Do We Need GNNs?**

Graphs are everywhere in the real world:
- **Social Networks**: Users (nodes) and friendships (edges).
- **Molecules**: Atoms (nodes) and bonds (edges).
- **Knowledge Graphs**: Entities (nodes) and relationships (edges).
- **Transportation Networks**: Locations (nodes) and routes (edges).
- **Recommender Systems**: Users and items as nodes, interactions as edges.

Traditional machine learning methods struggle to capture graph structure because:
1. Graphs are non-Euclidean, meaning they lack a regular grid-like structure.
2. Nodes and edges have varying numbers of connections.

GNNs overcome this by **propagating and learning information** through graph structures.

## **Step 3: Implement GNN Layers**
We perform **message passing** and a simple forward pass using matrix multiplication.

1. Add self-loops to the adjacency matrix.
2. Normalize the adjacency matrix.
3. Apply a linear transformation and activation function.

In [11]:
# Step 1: Add self-loops to the adjacency matrix
I = np.eye(A.shape[0])  # Identity matrix
A_hat = A + I  # Add self-loops

# Step 2: Normalize adjacency matrix
D = np.diag(np.sum(A_hat, axis=1))  # Degree matrix
D_inv_sqrt = np.linalg.inv(np.sqrt(D))  # D^(-1/2)
A_norm = D_inv_sqrt @ A_hat @ D_inv_sqrt  # Normalized adjacency matrix

# Step 3: Define a GNN Layer
def gnn_layer(X, A_norm, W):
    """
    Single GNN layer.
    Args:
        X: Input node features [N, F_in]
        A_norm: Normalized adjacency matrix [N, N]
        W: Weight matrix [F_in, F_out]
    Returns:
        Updated node features [N, F_out]
    """
    return np.maximum(0, A_norm @ X @ W)  # ReLU activation


## **Step 4: Build the GNN Model**
We use two GNN layers with ReLU activation to learn node embeddings and make predictions.


In [12]:
# Initialize weights for two GNN layers
np.random.seed(42)
W1 = np.random.randn(3, 4)  # Layer 1: input 3 -> output 4
W2 = np.random.randn(4, 2)  # Layer 2: input 4 -> output 2 (binary classification)

# Forward pass
H1 = gnn_layer(X, A_norm, W1)  # First GNN layer
H2 = gnn_layer(H1, A_norm, W2)  # Second GNN layer

# Softmax for predictions
def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

predictions = softmax(H2)
print("Node Predictions (Probabilities):\n", predictions)


Node Predictions (Probabilities):
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


## **Step 5: Loss Function and Training**
To train the GNN, implement a simple **cross-entropy loss** and a training loop using gradient descent.

In [13]:
# Cross-entropy loss
def cross_entropy(pred, labels):
    n = labels.shape[0]
    log_probs = -np.log(pred[range(n), labels])
    return np.sum(log_probs) / n

# Gradient descent
learning_rate = 0.01
for epoch in range(100):
    # Forward pass
    H1 = gnn_layer(X, A_norm, W1)
    H2 = gnn_layer(H1, A_norm, W2)
    pred = softmax(H2)

    # Compute loss
    loss = cross_entropy(pred, Y)

    # Backpropagation (manual gradients, simple approximation)
    grad_H2 = pred
    grad_H2[range(len(Y)), Y] -= 1
    grad_H2 /= len(Y)

    grad_W2 = H1.T @ (A_norm @ grad_H2)
    grad_H1 = A_norm @ (grad_H2 @ W2.T) * (H1 > 0)  # Backprop through ReLU
    grad_W1 = X.T @ (A_norm @ grad_H1)

    # Update weights
    W2 -= learning_rate * grad_W2
    W1 -= learning_rate * grad_W1

    # Print loss
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.6931
Epoch 10, Loss: 0.6931
Epoch 20, Loss: 0.6931
Epoch 30, Loss: 0.6931
Epoch 40, Loss: 0.6931
Epoch 50, Loss: 0.6931
Epoch 60, Loss: 0.6931
Epoch 70, Loss: 0.6931
Epoch 80, Loss: 0.6931
Epoch 90, Loss: 0.6931


## **Step 6: Evaluate the Model**

After training, check the predictions for each node:

In [14]:
# Final predictions
H1 = gnn_layer(X, A_norm, W1)
H2 = gnn_layer(H1, A_norm, W2)
pred = softmax(H2)
predicted_labels = np.argmax(pred, axis=1)

print("True Labels:      ", Y)
print("Predicted Labels: ", predicted_labels)


True Labels:       [0 1 0 1]
Predicted Labels:  [0 0 0 0]


## **How Do GNNs Work?**

GNNs use a **message-passing mechanism** to allow nodes to exchange information with their neighbors iteratively.

1. **Node Representation Initialization**:  
   Each node starts with initial features \( h_v^{(0)} \).

2. **Message Passing**:  
   At each iteration (or "layer"), a node aggregates information from its neighbors. This can be written as:
   \[
   h_v^{(t+1)} = \text{UPDATE}\left(h_v^{(t)}, \text{AGGREGATE}\left(\{h_u^{(t)} : u \in \mathcal{N}(v)\}\right)\right)
   \]
   - **AGGREGATE**: Combines information from the neighbors of \( v \) (e.g., using sum, mean, max, or attention mechanisms).
   - **UPDATE**: Updates the node's representation using the aggregated information (e.g., with a neural network).

3. **Final Representation**:  
   After \( T \) layers of message passing, nodes have learned **contextual embeddings** that encode both their features and the graph's structure.

4. **Graph-Level Representations**:  
   To produce a representation for the whole graph, a **pooling** operation (e.g., sum, mean, or max) combines all node representations.

## **Types of GNNs**

### 1. **Graph Convolutional Networks (GCN)**
- Generalizes traditional convolutions to graphs.
- Uses a normalized adjacency matrix to propagate information between nodes.

### 2. **Graph Attention Networks (GAT)**
- Uses **attention mechanisms** to assign different weights to neighboring nodes during aggregation.

### 3. **GraphSAGE**
- Scalable GNN for large graphs.
- Samples neighborhoods and aggregates information using mean, LSTM, or pooling.

### 4. **Message Passing Neural Networks (MPNNs)**
- A general framework where messages are passed between nodes through edges.

### 5. **Graph Isomorphism Networks (GIN)**
- A powerful model capable of distinguishing graph structures effectively.

## **Step 7: Define a Simple GCN Layer**

Each GCN layer aggregates information from neighboring nodes.

\[
H = \text{ReLU}(A_{\text{norm}} \cdot X \cdot W)
\]

- \( A_{\text{norm}} \): Normalized adjacency matrix.
- \( X \): Node features.
- \( W \): Weight matrix.
- ReLU activation: \( \max(0, x) \).

In [16]:
def gcn_layer(X, A_norm, W):
    """
    Single GCN layer: Aggregates node information.
    Args:
        X: Node features [N, F_in]
        A_norm: Normalized adjacency matrix [N, N]
        W: Weight matrix [F_in, F_out]
    Returns:
        Updated node features [N, F_out]
    """
    return np.maximum(0, A_norm @ X @ W)  # ReLU activation


## **Step 8: Build the GCN Model**

We define a 2-layer GCN for node classification.

In [17]:
# Initialize weight matrices
np.random.seed(42)
W1 = np.random.randn(3, 4)  # Layer 1: input 3 -> output 4
W2 = np.random.randn(4, 2)  # Layer 2: input 4 -> output 2 (binary classification)

# Forward pass through GCN
H1 = gcn_layer(X, A_norm, W1)  # First GCN layer
H2 = gcn_layer(H1, A_norm, W2)  # Second GCN layer

# Softmax activation for final predictions
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

predictions = softmax(H2)
print("Predictions (Node Probabilities):\n", predictions)


Predictions (Node Probabilities):
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


## **Step 9: Cross-Entropy Loss**

We compute the cross-entropy loss to measure the difference between predictions and labels.


In [18]:
def cross_entropy_loss(pred, labels):
    """
    Compute cross-entropy loss.
    Args:
        pred: Predicted probabilities [N, C]
        labels: Ground truth labels [N]
    """
    n = labels.shape[0]
    log_probs = -np.log(pred[range(n), labels])
    return np.sum(log_probs) / n

# Compute loss
loss = cross_entropy_loss(predictions, Y)
print("Cross-Entropy Loss:", loss)


Cross-Entropy Loss: 0.6931471805599453


## **Step 10: Training the GCN**

We train the GCN using gradient descent to minimize the loss.

In [19]:
# Training loop
learning_rate = 0.01
epochs = 100

for epoch in range(epochs):
    # Forward pass
    H1 = gcn_layer(X, A_norm, W1)
    H2 = gcn_layer(H1, A_norm, W2)
    pred = softmax(H2)

    # Compute loss
    loss = cross_entropy_loss(pred, Y)

    # Compute gradients (approximation for simplicity)
    grad_H2 = pred
    grad_H2[range(len(Y)), Y] -= 1
    grad_H2 /= len(Y)

    grad_W2 = H1.T @ (A_norm @ grad_H2)
    grad_H1 = A_norm @ (grad_H2 @ W2.T) * (H1 > 0)  # Backpropagate through ReLU
    grad_W1 = X.T @ (A_norm @ grad_H1)

    # Update weights
    W2 -= learning_rate * grad_W2
    W1 -= learning_rate * grad_W1

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.6931
Epoch 10, Loss: 0.6931
Epoch 20, Loss: 0.6931
Epoch 30, Loss: 0.6931
Epoch 40, Loss: 0.6931
Epoch 50, Loss: 0.6931
Epoch 60, Loss: 0.6931
Epoch 70, Loss: 0.6931
Epoch 80, Loss: 0.6931
Epoch 90, Loss: 0.6931


## **Step 11: Evaluate the Model**

After training, check the final predictions and compare them with true labels.

In [20]:
# Final predictions
H1 = gcn_layer(X, A_norm, W1)
H2 = gcn_layer(H1, A_norm, W2)
pred = softmax(H2)
predicted_labels = np.argmax(pred, axis=1)

print("True Labels:      ", Y)
print("Predicted Labels: ", predicted_labels)


True Labels:       [0 1 0 1]
Predicted Labels:  [0 0 0 0]


## **Applications of GNNs**

1. **Node Classification**: Predict node labels (e.g., categorizing users in social networks).
2. **Link Prediction**: Predict edges between nodes (e.g., friend recommendations).
3. **Graph Classification**: Classify entire graphs (e.g., molecular property prediction).
4. **Community Detection**: Identify groups of closely connected nodes.
5. **Recommender Systems**: Use user-item graphs to predict preferences.
6. **Knowledge Graph Completion**: Predict missing relationships between entities.
7. **Traffic Networks**: Analyze and predict traffic patterns in road networks.

## **Advantages of GNNs**

- GNNs can model both **node features** and **graph structures**.
- They work on irregular, non-Euclidean data like graphs.
- They allow nodes to learn from their **local neighborhoods**.

## **Popular Frameworks for GNNs**

- **PyTorch Geometric**: GNN library built on PyTorch.
- **Deep Graph Library (DGL)**: Flexible and scalable framework.
- **Spektral**: GNN library for TensorFlow/Keras.
- **NetworkX**: Graph processing library (non-deep learning).

In [1]:
pip install numpy


In [3]:
import numpy as np

# Adjacency Matrix (undirected graph with 4 nodes)
A = np.array([
    [0, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0]
])

# Node features (4 nodes, 3 features each)
X = np.array([
    [1, 2, 1],
    [2, 1, 0],
    [3, 1, 4],
    [4, 2, 1]
])

# Node labels (for classification, e.g., binary)
Y = np.array([0, 1, 0, 1])  # Ground truth labels


In [5]:
# Step 1: Add self-loops to the adjacency matrix
I = np.eye(A.shape[0])  # Identity matrix
A_hat = A + I  # Add self-loops

# Step 2: Normalize adjacency matrix
D = np.diag(np.sum(A_hat, axis=1))  # Degree matrix
D_inv_sqrt = np.linalg.inv(np.sqrt(D))  # D^(-1/2)
A_norm = D_inv_sqrt @ A_hat @ D_inv_sqrt  # Normalized adjacency matrix

# Step 3: Define a GNN Layer
def gnn_layer(X, A_norm, W):
    """
    Single GNN layer.
    Args:
        X: Input node features [N, F_in]
        A_norm: Normalized adjacency matrix [N, N]
        W: Weight matrix [F_in, F_out]
    Returns:
        Updated node features [N, F_out]
    """
    return np.maximum(0, A_norm @ X @ W)  # ReLU activation


In [6]:
# Initialize weights for two GNN layers
np.random.seed(42)
W1 = np.random.randn(3, 4)  # Layer 1: input 3 -> output 4
W2 = np.random.randn(4, 2)  # Layer 2: input 4 -> output 2 (binary classification)

# Forward pass
H1 = gnn_layer(X, A_norm, W1)  # First GNN layer
H2 = gnn_layer(H1, A_norm, W2)  # Second GNN layer

# Softmax for predictions
def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

predictions = softmax(H2)
print("Node Predictions (Probabilities):\n", predictions)


Node Predictions (Probabilities):
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


In [7]:
# Cross-entropy loss
def cross_entropy(pred, labels):
    n = labels.shape[0]
    log_probs = -np.log(pred[range(n), labels])
    return np.sum(log_probs) / n

# Gradient descent
learning_rate = 0.01
for epoch in range(100):
    # Forward pass
    H1 = gnn_layer(X, A_norm, W1)
    H2 = gnn_layer(H1, A_norm, W2)
    pred = softmax(H2)

    # Compute loss
    loss = cross_entropy(pred, Y)

    # Backpropagation (manual gradients, simple approximation)
    grad_H2 = pred
    grad_H2[range(len(Y)), Y] -= 1
    grad_H2 /= len(Y)

    grad_W2 = H1.T @ (A_norm @ grad_H2)
    grad_H1 = A_norm @ (grad_H2 @ W2.T) * (H1 > 0)  # Backprop through ReLU
    grad_W1 = X.T @ (A_norm @ grad_H1)

    # Update weights
    W2 -= learning_rate * grad_W2
    W1 -= learning_rate * grad_W1

    # Print loss
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.6931
Epoch 10, Loss: 0.6931
Epoch 20, Loss: 0.6931
Epoch 30, Loss: 0.6931
Epoch 40, Loss: 0.6931
Epoch 50, Loss: 0.6931
Epoch 60, Loss: 0.6931
Epoch 70, Loss: 0.6931
Epoch 80, Loss: 0.6931
Epoch 90, Loss: 0.6931
